In [1]:
import numpy as np
import pandas as pd

In [2]:
pip install lxml html5lib beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


## 1. Scraping the wikipedia data

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df = dfs[0]

In [4]:
df.head(6)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"


In [5]:
df = df[df['Borough'] != 'Not assigned']
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df.loc[df['Neighborhood']=='Not assigned','Neighborhood'] = df['Borough']
df

C:\Users\kazuya\.conda\envs\ds\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\kazuya\.conda\envs\ds\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
df.shape

(103, 3)

## 2. to make a data frame with geographical coordinates 

In [8]:
df2 = pd.read_csv('https://cocl.us/Geospatial_data')

In [9]:
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df2.shape

(103, 3)

In [11]:
dfm = pd.merge(df,df2,on="Postal Code")

In [12]:
dfm

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## 3. To clutering

In [13]:
from geopy.geocoders import Nominatim
import folium

In [14]:
address = 'North York'

geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('{} is {} , {}'.format(address,latitude,longitude))

North York is 43.7543263 , -79.44911696639593


### Check the location

In [21]:
map_ny = folium.Map(location=[latitude,longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfm['Latitude'], dfm['Longitude'], dfm['Borough'], dfm['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        color = 'blue',
        popup = label,
        fill = True,
        fill_color= '#3186cc',
        fill_opacity=0.7,
        parse_html = False
        ).add_to(map_ny)

map_ny

### Select data which contain the word Tront 

In [42]:
toront_data = dfm[dfm['Borough'].str.contains('Toront')].reset_index(drop=True)
toront_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [43]:
address = 'Toronto'

geolocator = Nominatim(user_agent='Tr_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('{} is {} , {}'.format(address,latitude,longitude))

Toronto is 43.6534817 , -79.3839347


In [45]:
map_Tront = folium.Map(location=[latitude,longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toront_data['Latitude'], toront_data['Longitude'], toront_data['Borough'], toront_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        color = 'blue',
        popup = label,
        fill = True,
        fill_color= '#3186cc',
        fill_opacity=0.7,
        parse_html = False
        ).add_to(map_Tront)
    
map_Tront

### Define Foursquare Credentials and Version

In [47]:
CLIENT_ID = 'YKTOJHFDTKGK4M0UF1YSR5OM3RWPKEI0THKBAJQG1ENSYTJU' # your Foursquare ID
CLIENT_SECRET = 'B4UR4GDRIVAO0SPUAOA5QZUI2BHJXZZLHBLWKICZE0JD0JJY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [48]:
toront_data.loc[0,'Neighborhood']

'Regent Park, Harbourfront'

In [49]:
tr_data_latitude = toront_data.loc[0,'Latitude']
tr_data_longitude = toront_data.loc[0,'Longitude']

tr_data_name = toront_data.loc[0,'Neighborhood']



create request URL

In [52]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    tr_data_latitude,
    tr_data_longitude,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YKTOJHFDTKGK4M0UF1YSR5OM3RWPKEI0THKBAJQG1ENSYTJU&client_secret=B4UR4GDRIVAO0SPUAOA5QZUI2BHJXZZLHBLWKICZE0JD0JJY&v=20180605&ll=43.6542599,-79.3606359&radius=500'

In [64]:
import json
import requests
from pandas.io.json import json_normalize

In [60]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eee23b60cc1fd001b47b30e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [61]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [73]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:,filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\kazuya\.conda\envs\ds\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Dominion Pub and Kitchen,Pub,43.656919,-79.358967


In [74]:
nearby_venues.shape

(30, 4)

create function to repeat above operate

In [83]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    LIMIT = 100
    venues_list = []
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        tr_data_latitude,
        tr_data_longitude,
        radius,
        LIMIT)
        
        results = requests.get(url).json()["response"]["groups"][0]['items']
        
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                   'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
        

In [84]:
toront_venues = getNearbyVenues(names = toront_data['Neighborhood'],
                               latitudes=toront_data['Latitude'],
                               longitudes=toront_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [86]:
toront_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


In [87]:
toront_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",30,30,30,30,30,30
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",30,30,30,30,30,30
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
Christie,30,30,30,30,30,30
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


make one hot encoding

In [111]:
toront_onehot = pd.get_dummies(toront_venues[['Venue Category']], prefix="", prefix_sep="")

toront_onehot['Neighborhood'] = toront_venues['Neighborhood']

fixed_columns = [toront_onehot.columns[-1]] + list(toront_onehot.columns[:-1])
toront_onehot = toront_onehot[fixed_columns]

toront_onehot

,Neighborhood,Bakery,Breakfast Spot,Café,Chocolate Shop,Coffee Shop,Dessert Shop,Distribution Center,Farmers Market,French Restaurant,Gym / Fitness Center,Historic Site,Park,Performing Arts Venue,Pub,Restaurant,Spa,Theater,Yoga Studio
0,"Regent Park, Harbourfront",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1166,"Business reply mail Processing Centre, South C...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1167,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1168,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [112]:
toront_grouped = toront_onehot.groupby('Neighborhood').mean().reset_index()

In [113]:
toront_grouped

,Neighborhood,Bakery,Breakfast Spot,Café,Chocolate Shop,Coffee Shop,Dessert Shop,Distribution Center,Farmers Market,French Restaurant,Gym / Fitness Center,Historic Site,Park,Performing Arts Venue,Pub,Restaurant,Spa,Theater,Yoga Studio
0,Berczy Park,0.066667,0.066667,0.066667,0.033333,0.2,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.1,0.033333,0.066667,0.033333,0.033333,0.066667,0.033333
1,"Brockton, Parkdale Village, Exhibition Place",0.066667,0.066667,0.066667,0.033333,0.2,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.1,0.033333,0.066667,0.033333,0.033333,0.066667,0.033333
2,"Business reply mail Processing Centre, South C...",0.066667,0.066667,0.066667,0.033333,0.2,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.1,0.033333,0.066667,0.033333,0.033333,0.066667,0.033333
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.066667,0.066667,0.066667,0.033333,0.2,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.1,0.033333,0.066667,0.033333,0.033333,0.066667,0.033333
4,Central Bay Street,0.066667,0.066667,0.066667,0.033333,0.2,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.1,0.033333,0.066667,0.033333,0.033333,0.066667,0.033333
5,Christie,0.066667,0.066667,0.066667,0.033333,0.2,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.1,0.033333,0.066667,0.033333,0.033333,0.066667,0.033333
6,Church and Wellesley,0.066667,0.066667,0.066667,0.033333,0.2,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.1,0.033333,0.066667,0.033333,0.033333,0.066667,0.033333
7,"Commerce Court, Victoria Hotel",0.066667,0.066667,0.066667,0.033333,0.2,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.1,0.033333,0.066667,0.033333,0.033333,0.066667,0.033333
8,Davisville,0.066667,0.066667,0.066667,0.033333,0.2,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.1,0.033333,0.066667,0.033333,0.033333,0.066667,0.033333
9,Davisville North,0.066667,0.066667,0.066667,0.033333,0.2,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.1,0.033333,0.066667,0.033333,0.033333,0.066667,0.033333


In [90]:
toront_grouped.shape

(39, 19)

create function to choose top number of venues

In [92]:
def return_most_common_venues(row,num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [104]:
num_top_venues = 10

indicators = ['st','nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind + 1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neigh_venues_sorted = pd.DataFrame(columns = columns)
neigh_venues_sorted['Neighborhood'] = toront_grouped['Neighborhood']

for ind in np.arange(toront_grouped.shape[0]):
    neigh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toront_grouped.iloc[ind,:], num_top_venues)
    
neigh_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
4,Central Bay Street,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop


### Clustering

In [105]:
from sklearn.cluster import KMeans

In [106]:
kclusters = 5

toront_clustering = toront_grouped.drop('Neighborhood',1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toront_clustering)

kmeans.labels_[0:10]

C:\Users\kazuya\.conda\envs\ds\lib\site-packages\ipykernel_launcher.py:5: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  """


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [107]:
neigh_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)

toront_merged = toront_data

toront_merged = toront_merged.join(neigh_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toront_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop


In [109]:
toront_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Distribution Center,Chocolate Shop,Dessert Shop


Visualize the sulting cluter

In [117]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [118]:
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []

for lat, lon, poi, cluster in zip(toront_merged['Latitude'], toront_merged['Longitude'], toront_merged['Neighborhood'], toront_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius = 5,
        popup=label,
        color=rainbow[cluster-1],
        fill = True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters